# Assignment 2

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./05_src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model]() or a linear model like [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) or [Ridge Regressors](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html).
        - An advanced regressor of your choice (e.g., Bagging, Boosting, SVR, etc.). TIP: select a tree-based method such that it does not take too long to run SHAP further below. 

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Place the files in the ../../05_src/data/fires/ directory and load the appropriate file. 

In [1]:
# Load the libraries as required.
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error


In [2]:
# Load data
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../../05_src/data/fires/forestfires.csv', header = 0, names = columns))
fires_dt.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   month    517 non-null    object 
 3   day      517 non-null    object 
 4   ffmc     517 non-null    float64
 5   dmc      517 non-null    float64
 6   dc       517 non-null    float64
 7   isi      517 non-null    float64
 8   temp     517 non-null    float64
 9   rh       517 non-null    int64  
 10  wind     517 non-null    float64
 11  rain     517 non-null    float64
 12  area     517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


In [3]:
fires_dt.head()
#fires_dt.shape

,coord_x,coord_y,month,day,ffmc,dmc,dc,isi,temp,rh,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


# Get X and Y

Create the features data frame and target data.

In [4]:
X = fires_dt.drop(columns=["area"]) # features
print(X.shape)

(517, 12)


In [5]:
y = fires_dt["area"] # Target data
print(y.shape)

(517,)


# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [6]:
#columns definition
numeric_cols = [
    'coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi',
    'temp', 'rh', 'wind', 'rain'
]

categorical_cols = ['month', 'day']

std_scaler = StandardScaler()
onehot = OneHotEncoder(handle_unknown='ignore')

preproc1 = ColumnTransformer([
    ("num", std_scaler, numeric_cols),
    ("cat", onehot, categorical_cols)
])

### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [7]:
# Logaritmic transformation
log_transform = FunctionTransformer(np.log1p, validate=False)

# Pipeline for numeric variables
numeric_pipeline = Pipeline([
    ("log", log_transform),
    ("scaler", std_scaler)
])

preproc2 = ColumnTransformer([
    ("num", numeric_pipeline, numeric_cols),
    ("cat", onehot, categorical_cols)
])

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [8]:
# Models definition
baseline_model = Ridge()
advanced_model = RandomForestRegressor(random_state=42)

In [9]:
# Pipeline A = preproc1 + baseline
pipeline_A = Pipeline([
    ("preprocessing", preproc1),
    ("regressor", baseline_model)
])
pipeline_A

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['month', 'day'])])),
                ('regressor', Ridge())])

In [10]:
# Pipeline B = preproc2 + baseline
pipeline_B = Pipeline([
    ("preprocessing", preproc2),
    ("regressor", baseline_model)
])
pipeline_B

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('log',
                                                                   FunctionTransformer(func=<ufunc 'log1p'>)),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['month', 'day'])])),
                ('regressor', Ridge())])

In [11]:
# Pipeline C = preproc1 + advanced model
pipeline_C = Pipeline([
    ("preprocessing", preproc1),
    ("regressor", advanced_model)
])
pipeline_C

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['month', 'day'])])),
                ('regressor', RandomForestRegressor(random_state=42))])

In [12]:
# Pipeline D = preproc2 + advanced model
pipeline_D = Pipeline([
    ("preprocessing", preproc2),
    ("regressor", advanced_model)
])
pipeline_D

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('log',
                                                                   FunctionTransformer(func=<ufunc 'log1p'>)),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['month', 'day'])])),
                ('regressor', RandomForestRegressor(random_state=42))])

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [13]:
#Pipeline A
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Define hyperparameter grid for Ridge
param_grid_A = {
    "regressor__alpha": [0.01, 0.1, 1.0, 10.0]
}

# Create GridSearchCV object
grid_A = GridSearchCV(
    pipeline_A,              # pipeline with preproc1 + Ridge
    param_grid=param_grid_A, # hyperparameter space
    cv=5,                    # 5-fold cross-validation
    scoring="neg_mean_absolute_error",  # evaluation metric
    n_jobs=-1               
)

# Fit the model to training data
grid_A.fit(X_train, y_train)

# Best cross-validated results
print("Best MAE (CV)- pipeline_A:", -grid_A.best_score_)
print("Best params - pipeline_A:", grid_A.best_params_)

Best MAE (CV)- pipeline_A: 17.723576862492802
Best params - pipeline_A: {'regressor__alpha': 10.0}


In [14]:
#Pipeline B

# Define hyperparameter grid for Ridge in pipeline_B
param_grid_B = {
    "regressor__alpha": [0.01, 0.1, 1.0, 10.0]
}

# Create GridSearchCV object for pipeline_B
grid_B = GridSearchCV(
    pipeline_B,             # pipeline with preproc2 + Ridge
    param_grid=param_grid_B,
    cv=5,
    scoring="neg_mean_absolute_error", 
    n_jobs=-1
)

# Fit to training data
grid_B.fit(X_train, y_train)

# Best cross-validated results
print("Best MAE (CV) - pipeline_B:", -grid_B.best_score_)
print("Best params - pipeline_B:", grid_B.best_params_)


Best MAE (CV) - pipeline_B: 17.66418553843956
Best params - pipeline_B: {'regressor__alpha': 10.0}


In [15]:
#Pipeline C

# Define hyperparameter grid for Random Forest in pipeline_C
param_grid_C = {
    "regressor__n_estimators": [50, 100],         # Number of trees in the forest
    "regressor__max_depth": [5, 10]               # Maximum depth of each tree
}

# Create GridSearchCV object for pipeline_C
grid_C = GridSearchCV(
    pipeline_C,             # pipeline with preproc1 + Random Forest
    param_grid=param_grid_C,
    cv=5,
    scoring="neg_mean_absolute_error", 
    n_jobs=-1
)

# Fit the grid search on training data
grid_C.fit(X_train, y_train)

# Best results
print("Best MAE (CV) - pipeline_C:", -grid_C.best_score_)
print("Best params - pipeline_C:", grid_C.best_params_)


Best MAE (CV) - pipeline_C: 19.069211985106932
Best params - pipeline_C: {'regressor__max_depth': 5, 'regressor__n_estimators': 100}


In [16]:
#Pipeline D

# Define hyperparameter grid for Random Forest in pipeline_D
param_grid_D = {
    "regressor__n_estimators": [50, 100],   # Number of trees
    "regressor__max_depth": [5, 10]         # Maximum depth of each tree
}

# Create GridSearchCV object
grid_D = GridSearchCV(
    pipeline_D,                 # pipeline with preproc2 + Random Forest
    param_grid=param_grid_D,
    cv=5,
    scoring="neg_mean_absolute_error",
    n_jobs=-1
)

# Fit to training data
grid_D.fit(X_train, y_train)

# Print best cross-validated results
print("Best MAE (CV) - pipeline_D:", -grid_D.best_score_)
print("Best params - pipeline_D:", grid_D.best_params_)


Best MAE (CV) - pipeline_D: 19.13242279682926
Best params - pipeline_D: {'regressor__max_depth': 5, 'regressor__n_estimators': 100}


# Evaluate

+ Which model has the best performance?

In [17]:
# Evaluate all best models on the test set
models = {
    "pipeline_A": grid_A.best_estimator_,
    "pipeline_B": grid_B.best_estimator_,
    "pipeline_C": grid_C.best_estimator_,
    "pipeline_D": grid_D.best_estimator_
}

print("Test MAE for each pipeline with test set:")
for name, model in models.items():
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    print(f"{name}: {mae:.2f}")


Test MAE for each pipeline with test set:
pipeline_A: 24.12
pipeline_B: 24.07
pipeline_C: 26.03
pipeline_D: 25.75


### Evaluation: Best Performing Model

The following table summarizes the **Mean Absolute Error (MAE)** from both **cross-validation (CV)** and the **final test set** evaluation for each pipeline:

| Pipeline   | Preprocessing | Model              | MAE (CV) ↓ | MAE (Test) ↓ | Best Params |
|------------|---------------|--------------------|------------|--------------|-------------|
| A          | `preproc1`    | `Ridge`            | 17.72      | 24.12        | `alpha=10.0` |
| B          | `preproc2`    | `Ridge`            | **17.66** ✅ | **24.07** ✅ | `alpha=10.0` |
| C          | `preproc1`    | `RandomForest`     | 19.07      | 26.03        | `max_depth=5`, `n_estimators=100` |
| D          | `preproc2`    | `RandomForest`     | 19.13      | 25.75        | `max_depth=5`, `n_estimators=100` |

> ✅ **Conclusion:**  
> Based on both cross-validation and test set performance, **Pipeline B** is the best-performing model.  
> It combines a **log-transformed preprocessor (`preproc2`)** with a **regularized linear model (Ridge)**, achieving the **lowest MAE on both CV (17.66)** and **test set (24.07)**.  
> This suggests that the log transformation improved the numeric feature distribution and the Ridge model was well-suited for this prediction task.

# Export

+ Save the best performing model to a pickle file.

In [18]:
import pickle

# Export the best-performing model (pipeline_B from grid_B)
with open("forestfires_model_pipeline_B.pkl", "wb") as f:
    pickle.dump(grid_B.best_estimator_, f)

print("Model exported successfully.")

Model exported successfully.


# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

*(Answer here.)*

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at the `help` channel. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.